In [ ]:
import pandas as pd
import numpy as np

df = pd.read_pickle("LSWMD.pkl")
df = df.drop(['waferIndex'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        811457 non-null  object 
 1   dieSize         811457 non-null  float64
 2   lotName         811457 non-null  object 
 3   trianTestLabel  811457 non-null  object 
 4   failureType     811457 non-null  object 
dtypes: float64(1), object(4)
memory usage: 31.0+ MB


In [131]:
def find_dim(x):
    dim0=np.size(x,axis=0)
    dim1=np.size(x,axis=1)
    return dim0,dim1

df['waferMapDim']=df.waferMap.apply(find_dim)

df.sample(5)

,waferMap,dieSize,lotName,trianTestLabel,failureType,waferMapDim
570900,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2,...",710.0,lot35751,[],[],"(32, 29)"
453176,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",733.0,lot27795,[],[],"(31, 31)"
454510,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,...",733.0,lot27959,[],[],"(31, 31)"
807077,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",1060.0,lot47362,[[Test]],[[none]],"(41, 33)"
163638,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",14116.0,lot10507,[],[],"(212, 84)"


In [132]:
import numpy as np

def unwrap_nested_array(x):
    try:
        if isinstance(x, (list, np.ndarray)) and len(x) > 0:
            if isinstance(x[0], (list, np.ndarray)) and len(x[0]) > 0:
                return x[0][0]
    except:
        pass
    return None

# 문자열 추출
df['failureStr'] = df['failureType'].apply(unwrap_nested_array)
df['trainTestStr'] = df['trianTestLabel'].apply(unwrap_nested_array)

# 매핑 정의
mapping_type = {
    'Center': 0, 'Donut': 1, 'Edge-Loc': 2, 'Edge-Ring': 3,
    'Loc': 4, 'Random': 5, 'Scratch': 6, 'Near-full': 7, 'none': 8
}
mapping_traintest = {'Training': 0, 'Test': 1}

# 매핑 적용
df['failureNum'] = df['failureStr'].map(mapping_type)
df['trainTestNum'] = df['trainTestStr'].map(mapping_traintest)
df.drop(['failureStr', 'trainTestStr'], axis=1, inplace=True)
df.head()

,waferMap,dieSize,lotName,trianTestLabel,failureType,waferMapDim,failureNum,trainTestNum
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],"(45, 48)",8.0,0.0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],"(45, 48)",8.0,0.0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],"(45, 48)",8.0,0.0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],"(45, 48)",8.0,0.0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],"(45, 48)",8.0,0.0


In [133]:
import numpy as np
from tqdm import tqdm  # 진행률 바

sub_df = df.loc[df['waferMapDim'] == (26, 26)]
sub_wafer = sub_df['waferMap'].values

sw = np.ones((1, 26, 26))
label = []

print("Preprocessing wafer maps...")

for i in tqdm(range(len(sub_df)), desc="Processing", unit="sample"):
    row = sub_df.iloc[i, :]

    # skip null label
    if len(row['failureType']) == 0:
        continue

    # wafer image 추가
    sw = np.concatenate((sw, row['waferMap'].reshape(1, 26, 26)))

    # 라벨 추가
    label.append(row['failureType'][0][0])  # numpy.ndarray 구조 대응

# 최종 배열 생성
x = sw[1:]
y = np.array(label).reshape((-1, 1))

print('x shape : {}, y shape : {}'.format(x.shape, y.shape))


Preprocessing wafer maps...


Processing: 100%|██████████| 30078/30078 [02:37<00:00, 191.53sample/s] 

x shape : (14366, 26, 26), y shape : (14366, 1)


In [134]:
x = x.reshape((-1, 26, 26, 1))
faulty_case = np.unique(y)
print('Faulty case list : {}'.format(faulty_case))

Faulty case list : ['Center' 'Donut' 'Edge-Loc' 'Edge-Ring' 'Loc' 'Near-full' 'Random'
 'Scratch' 'none']


In [135]:
for f in faulty_case :
    print('{} : {}'.format(f, len(y[y==f])))

Center : 90
Donut : 1
Edge-Loc : 296
Edge-Ring : 31
Loc : 297
Near-full : 16
Random : 74
Scratch : 72
none : 13489


In [136]:
new_x = np.zeros((len(x), 26, 26, 3))

for w in range(len(x)):
    for i in range(26):
        for j in range(26):
            new_x[w, i, j, int(x[w, i, j])] = 1

new_x.shape

/tmp/ipykernel_196016/843030097.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_x[w, i, j, int(x[w, i, j])] = 1


(14366, 26, 26, 3)

In [137]:
import torch

label_mapping = {
    'Center': 0, 'Donut': 1, 'Edge-Loc': 2, 'Edge-Ring': 3,
    'Loc': 4, 'Random': 5, 'Scratch': 6, 'Near-full': 7, 'none': 8
}

y_int = [label_mapping[label[0]] for label in y]  
y_tensor = torch.tensor(y_int, dtype=torch.long) 
x_tensor = torch.tensor(np.transpose(new_x, (0, 3, 1, 2)), dtype=torch.float32) 

print(x_tensor.shape)
print(y_tensor.shape)

print("y 원소 예시:", y[0])      
print("y 정수 인코딩:", y_int[0])  
print("y_tensor shape:", y_tensor.shape)  

torch.Size([14366, 3, 26, 26])
torch.Size([14366])
y 원소 예시: ['none']
y 정수 인코딩: 8
y_tensor shape: torch.Size([14366])


In [138]:
import torch
import torch.nn as nn

class WaferCNN(nn.Module):
    def __init__(self, num_classes=9):
        super(WaferCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # 3 input channels → 64
            nn.ReLU(),
            nn.MaxPool2d(2),  # 26x26 → 13x13

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # 64 input → 128 output
            nn.ReLU(),
            nn.MaxPool2d(2),  # 13x13 → 6x6
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# 1. Dataset & DataLoader
dataset = TensorDataset(x_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 2. 모델, 손실 함수, 옵티마이저
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WaferCNN(num_classes=9).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [142]:
# # 3. 학습 루프
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)  
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataset)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Acc: {accuracy:.6f}")

Epoch [1/40] Loss: 0.0069 Acc: 0.997773
Epoch [2/40] Loss: 0.0058 Acc: 0.998051
Epoch [3/40] Loss: 0.0065 Acc: 0.997842
Epoch [4/40] Loss: 0.0057 Acc: 0.997842
Epoch [5/40] Loss: 0.0079 Acc: 0.997424
Epoch [6/40] Loss: 0.0076 Acc: 0.997146
Epoch [7/40] Loss: 0.0095 Acc: 0.996520
Epoch [8/40] Loss: 0.0092 Acc: 0.996728
Epoch [9/40] Loss: 0.0052 Acc: 0.998260
Epoch [10/40] Loss: 0.0050 Acc: 0.998329
Epoch [11/40] Loss: 0.0068 Acc: 0.997564
Epoch [12/40] Loss: 0.0063 Acc: 0.997633
Epoch [13/40] Loss: 0.0068 Acc: 0.997564
Epoch [14/40] Loss: 0.0046 Acc: 0.998399
Epoch [15/40] Loss: 0.0063 Acc: 0.997703
Epoch [16/40] Loss: 0.0049 Acc: 0.998121
Epoch [17/40] Loss: 0.0103 Acc: 0.996589
Epoch [18/40] Loss: 0.0050 Acc: 0.998329
Epoch [19/40] Loss: 0.0051 Acc: 0.997981
Epoch [20/40] Loss: 0.0037 Acc: 0.998608
Epoch [21/40] Loss: 0.0039 Acc: 0.998329
Epoch [22/40] Loss: 0.0097 Acc: 0.996520
Epoch [23/40] Loss: 0.0056 Acc: 0.997633
Epoch [24/40] Loss: 0.0044 Acc: 0.998260
Epoch [25/40] Loss: 0.006